In [1]:
import numpy as np
from scipy.optimize import minimize

In [9]:
def read_csv_data(filename):
    data = np.loadtxt(filename, delimiter=',', skiprows=1)
    x = data[:,0]
    y = data[:,1]
    n = len(x)
    return x, y, n

In [10]:
def assign_t(n):
    return np.linspace(6, 60, n)

In [11]:
def xy_model(t, theta, M, X):
    x_model = t * np.cos(theta) - np.exp(M * np.abs(t)) * np.sin(0.3 * t) * np.sin(theta) + X
    y_model = 42 + t * np.sin(theta) + np.exp(M * np.abs(t)) * np.sin(0.3 * t) * np.cos(theta)
    return x_model, y_model

In [12]:
def error(params, x, y, t):
    theta, M, X = params
    x_m, y_m = xy_model(t, theta, M, X)
    residuals = ((x - x_m) ** 2 + (y - y_m) ** 2)
    return np.sum(residuals)

In [18]:

def compute_gradients(params, x, y, t, h=1e-6):
    grads = np.zeros_like(params)
    base_error = error(params, x, y, t)
    for i in range(len(params)):
        params_step = params.copy()
        params_step[i] += h
        grads[i] = (error(params_step, x, y, t) - base_error) / h
    return grads

# Gradient descent optimization
params = np.array([np.radians(25), 0.0, 50.0])  # Initial guess
learning_rate = 0.01
max_iters = 1000

# Parameter bounds
bounds = [ (np.radians(0), np.radians(50)),  # theta
           (-0.05, 0.05),                   # M
           (0, 100) ]                       # X

for it in range(max_iters):
    grads = compute_gradients(params, x, y, t)
    params = params - learning_rate * grads
    # Clip to bounds
    for i, (lo, hi) in enumerate(bounds):
        params[i] = np.clip(params[i], lo, hi)
    this_error = error(params, x, y, t)
    if it % 100 == 0:
        print(f"Iteration {it}: error={this_error:.2e}, params={params}")

print(f"Estimated: theta={np.degrees(params[0]):.2f} deg, M={params[1]:.5f}, X={params[2]:.2f}")

Iteration 0: error=3.00e+06, params=[ 8.72664626e-01 -5.00000000e-02  1.00000000e+02]
Iteration 100: error=4.91e+06, params=[0.e+00 5.e-02 1.e+02]
Iteration 200: error=4.91e+06, params=[0.e+00 5.e-02 1.e+02]
Iteration 300: error=4.91e+06, params=[0.e+00 5.e-02 1.e+02]
Iteration 400: error=4.91e+06, params=[0.e+00 5.e-02 1.e+02]
Iteration 500: error=4.91e+06, params=[0.e+00 5.e-02 1.e+02]
Iteration 600: error=4.91e+06, params=[0.e+00 5.e-02 1.e+02]
Iteration 700: error=4.91e+06, params=[0.e+00 5.e-02 1.e+02]
Iteration 800: error=4.91e+06, params=[0.e+00 5.e-02 1.e+02]
Iteration 900: error=4.91e+06, params=[0.e+00 5.e-02 1.e+02]
Estimated: theta=50.00 deg, M=-0.05000, X=0.00


In [21]:
if __name__ == '__main__':
    x, y, n = read_csv_data('xy_data.csv')
    t = assign_t(n)
    bounds = [ (np.radians(0), np.radians(50)),  # theta in radians
               (-0.05, 0.05),                  # M
               (0, 100) ]                      # X
    guess = [ np.radians(25), 0.0, 50.0 ]      # initial weights
    result = minimize(error, guess, args=(x, y, t), bounds=bounds)
    print(result.x)  # Estimated [theta, M, X]
    print(f"Final error for minimize: {result.fun:.2e}")
    #theta is given in radians need to be converted to degrees

[ 5.1628954e-01 -5.0000000e-02  5.5011614e+01]
Final error for minimize: 7.72e+05


In [22]:
from scipy.optimize import curve_fit

def xy_model_for_curve_fit(t, theta, M, X):
    x_model, y_model = xy_model(t, theta, M, X)
    return np.concatenate((x_model, y_model))

# Flatten the observed x and y data for curve_fit
observed_data_flat = np.concatenate((x, y))

initial_guess = [np.radians(25), 0.0, 50.0]


lower_bounds = [bounds[0][0], bounds[1][0], bounds[2][0]]
upper_bounds = [bounds[0][1], bounds[1][1], bounds[2][1]]



popt, pcov = curve_fit(xy_model_for_curve_fit,
                       t,
                       observed_data_flat,
                       p0=initial_guess,
                       bounds=(lower_bounds, upper_bounds))

print(f"Estimated parameters using curve_fit: [theta_rad, M, X] = {popt}")
print(f"Estimated theta: {np.degrees(popt[0]):.2f} degrees")
print(f"Estimated M: {popt[1]:.5f}")
print(f"Estimated X: {popt[2]:.2f}")

final_curve_fit_error = error(popt, x, y, t)
print(f"Final error for curve_fit: {final_curve_fit_error:.2e}")

Estimated parameters using curve_fit: [theta_rad, M, X] = [ 5.16308288e-01 -4.99999997e-02  5.50134594e+01]
Estimated theta: 29.58 degrees
Estimated M: -0.05000
Estimated X: 55.01
Final error for curve_fit: 7.72e+05
